This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [3]:
%%capture
%pip install python-dotenv
# couldn't do so in environment for some reason, added to code itself

In [4]:
# get api key from your .env file
import os
from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

print(API_KEY)

MzVs-x8pcz_Azvs9jzSG


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [5]:
# First, import the relevant modules
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [20]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
def fetch_nasdaq_dataset(database, dataset, parameters={}):
    r = requests.get(
        f'https://data.nasdaq.com/api/v3/datasets/{database}/{dataset}/data.json', 
        params={**parameters, 'API_KEY': API_KEY}
    )
    return r.json()

test_query_results = fetch_nasdaq_dataset('FSE', 'AFX_X', {
    'start_date': '2014-01-01',
    'end_date': '2014-12-31',
})

In [21]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
if test_query_results:
    display(test_query_results['dataset_data']['data'][0])
else:
    display('NO DATA RETURNED')

['2014-12-30',
 21.27,
 21.27,
 20.97,
 21.1,
 None,
 42065.0,
 888369.0,
 None,
 None,
 None]

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [23]:
AFX_X_data = fetch_nasdaq_dataset('FSE', 'AFX_X',{
    'start_date': '2017-01-01',
    'end_date': '2017-12-31',
})
display(AFX_X_data['dataset_data']['column_names'])

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [45]:
AFX_X_prices = AFX_X_data['dataset_data']['data']

def get_median(lst):
    sorted_lst = sorted(lst)
    midpoint = round(len(sorted_lst) / 2)
    
    if len(sorted_lst) % 2 == 0:
        return (sorted_lst[midpoint] + sorted_lst[midpoint - 1]) / 2
    else:
        return sorted_lst[midpoint]
    
metrics = {
    'highest_open_price': max([ x[1] for x in AFX_X_prices if x[1] ]),
    'lowest_open_price': min([ x[2] for x in AFX_X_prices if x[2] ]),
    'largest_change_in_one_day': round(max([ 
        abs(x[4] - x[1]) 
        for x in AFX_X_prices 
        if x[1] and x[4] 
    ]), 2),
    'largest_change_in_closing_prices_between_two_days': round(max([
        abs(p[0] - p[1]) for p in
        zip([ x[4] for x in AFX_X_prices ][:-1], [ x[4] for x in AFX_X_prices[1:] ])
    ]), 2),
    'average_traded_volume': sum([ x[6] for x in AFX_X_prices ]) / len([ x[6] for x in AFX_X_prices if x[6] ]),
    'median_traded_volume': get_median([ x[6] for x in AFX_X_prices if x[6] ]),
}

metrics

{'highest_open_price': 53.11,
 'lowest_open_price': 34.12,
 'largest_change_in_one_day': 1.69,
 'largest_change_in_closing_prices_between_two_days': 2.56,
 'average_traded_volume': 89124.33725490196,
 'median_traded_volume': 76600.0}